# Deploying A Schedule Building Algorithm
> Automating a manual process!

- toc:true
- badges: true
- comments: true
- author: David De Sa
- categories: [jupyter]

# Context
## Goal
Deploy an algorithm for schedulers to use that is:

-easy to use and learn

-transparent

-quick

-flexible

## Motivation
In a 24/7 manufacturing environment, the weekend shifts are covered mostly by overtime, which is scheduled according to employee availability, subject to constraints outlined in the labour collective agreement. Due to changing production needs as well as staff availability, the schedule must be re-drafted many times, often on short notice and under tight time constraints. Drafting it is tedious, error-prone, and time consuming. It could be automated.

## Challenges
-Data being manually entered in a variety of formats or not available in machine readable form. (e.g. total hours, employee type, employee availability, individualized job restrictions, outlier reasons for non-eligibility such as consecutive days worked) This is probably the main challenge!!

-Algorithm ambiguity. The collective agreement defines the constraints that each assignment decision is subject to, but doesn't strictly specify all aspects, allowing for arbitrary choice on schedulers part

-Many esoteric rules and edge cases around assignments being valid or not, which are also subject to change at time of contract renegotiation.

-Usability. The deployment must be available to all schedulers, and have a very low barrier to entry w.r.t. training and usability.

## A Bit Of Lore
The notion of automating the process solution has been bouncing around my head for over a year. I always felt the main challenge was the situation posed by the data... the bad formatting relegated to excel sheets, necessarily made that way from human input and usage modality not being the same as what is best for machine readability. I am very confident I could make something that worked in VBA, but the nature of that language makes it such a pain to develop with, particularly with bad formatting. I knew python was a better solution, but didn't have the bridge between the two to make something that worked. Finally when following the FastAI course I came across the HFS+Gradio wombo combo for sharing python scripts publicly via a great UI. This discovery got me to finally choose to commit to that solution path

# Solution
## Codebase
Using Gradio hosted via HFS for making a python algorithm available with easy integration of inputs and outputs. At first blush I thought that Pandas DataFrames would be the best input mode for tabular data, but ruled that out when HFS didn't allow for bulk copying and pasting. Maybe that was for the best because this pushed me to figure out how to work with the generic File input/output mode. It might be a little more painful to program (have to define methods to identify the right tables within the excel file), but a lot nice on the end user experience (drag and drop relevant files and go!). I was concerned about the extra steps of processing the excel file, but as with everything Python there is a library for that! I started off with some basic tests to ensure that what I wanted/needed to do was possible.

### File Manipulation
Here is my proof of concept for File manipulation. If copied into an empty Gradio space on HFS, it takes in an excel file, and adds a new table to the spreadsheet. This was all i needed to know that this could be done...

In [ ]:
import gradio as gr
import openpyxl as pyxl #openPyXl allows for excel file manipulation in python

def myFunction(fl,txt):
    myWb=pyxl.load_workbook(fl.name) #Load excel file
    tab = pyxl.worksheet.table.Table(displayName="Table3", ref="E1:F5") #Define new table
    style = pyxl.worksheet.table.TableStyleInfo(name="TableStyleMedium9",showRowStripes=True, showColumnStripes=True)
    tab.tableStyleInfo = style #Assign style to table
    ws=myWb.active 
    ws.add_table(tab) #Add defined table to sheet within the loaded workbook
    otpt_fl_name='try.xlsx' 
    myWb.save(otpt_fl_name) #Save file
    return otpt_fl_name #Define output for HFS interface

demo = gr.Interface(
    myFunction, #Func to take in file and text
    [
        gr.File(
        ),
        gr.Textbox(
            label="Initial text",
            lines=3,
            value="The quick brown fox jumped over the lazy dogs.",
        ),
    ],
    gr.File(),
    description="Enter refusal files",
)
demo.launch()

### Retrieving Disparate Tables
As mentioned previously, one challenge would be to pull data from tables scattered in an unpredictable way throughout the sheet. Here I had to remember that sometimes the easiest way to rob a bank is through the front door, not trying to break through the wall... I simply changed the existing excel template files (filled in by end user) so that the data tables were actually defined as 'Tables' by excel... this made them reference-able by the openPyXl tools. Some further data type transformations were required. Example with a blank book containing a trivial data table called 'tstTbl' in Excel:

In [ ]:
import openpyxl as pyxl
import pandas as pd
import numpy as np
myWb=pyxl.load_workbook('/images/Other_Files/TblTestBook.xlsx')
myWb['Sheet1'].tables['tstTbl'] #Pull out table